In [ ]:
# Add the parent directory to the path
import sys, os
sys.path.insert(0, os.path.abspath("../.."))

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

import torch
import random
from tabulate import tabulate

from src.data.config import HP

from src.solvers.spnia_asym import AsymmetricSPNI
from src.models.ShortestPathGrb import shortestPathGrb
from src.models.ShortestPathGrid import ShortestPathGrid
from src.solvers.BendersDecomposition import BendersDecomposition
from src.models.CalibrationTrainer import CalibratorTrainer

from src.scripts.compare_po_spo import compare_po_spo
from src.scripts.setup import gen_data, gen_train_data, setup_po_model, setup_spo_model, setup_hybrid_spo_model

In [ ]:
# # Define hyperparameters
# B = 5
# network = (6, 8)
# random_seed = 31
# intd_seed = 53
# data_loader_seed = 17

# # ML hyperparameters
# num_features = 5
# num_train_samples = 600
# validation_size = 100
# num_test_samples = 200
# data_loader_batch_size = 32
# po_epochs = 60
# spo_epochs = 20
# po_lr = 1e-2
# spo_lr = 1e-3
# lam = .15
# deg = 7
# anchor = "mse"
# spo_po_epochs = 0
# noise_width = 0.05

# # Interdictor parameters
# benders_max_count = 50
# benders_eps = 1e-3
# lsd = 1e-5



# Initialize the configuration class
cfg = HP()

# Set the random seed for reproducibility
np.random.seed(cfg.get("random_seed"))
random.seed(cfg.get("random_seed"))
torch.manual_seed(cfg.get("random_seed"))
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(cfg.get("random_seed"))

# # Change configuration parameters (optional)
# cfg.set("budget", B)
# cfg.set("grid_size", network)
# cfg.set("random_seed", random_seed)
# cfg.set("intd_seed", intd_seed)
# cfg.set("data_loader_seed", data_loader_seed)

# cfg.set("num_features", num_features)
# cfg.set("num_train_samples", num_train_samples)
# cfg.set("num_test_samples", num_test_samples)
# cfg.set("validation_size", validation_size)
# cfg.set("data_loader_batch_size", data_loader_batch_size)
# cfg.set("po_epochs", po_epochs)
# cfg.set("spo_epochs", spo_epochs)
# cfg.set("spo_po_epochs", spo_po_epochs)
# cfg.set("po_lr", po_lr)
# cfg.set("spo_lr", spo_lr)
# cfg.set("lam", lam)
# cfg.set("anchor", anchor)
# cfg.set("deg", deg)
# cfg.set("noise_width", noise_width)

# cfg.set("benders_max_count", benders_max_count)
# cfg.set("benders_eps", benders_eps)
# cfg.set("lsd", lsd)


In [ ]:
cfg.get("grid_size")

In [ ]:
# Define a graph with appropriate dimensions and an opt_model 
# for solving the shortest path problem on the graph
m,n = cfg.get("grid_size")
graph = ShortestPathGrid(m,n)
opt_model = shortestPathGrb(graph)

# Generate normalized training and testing data
training_data, testing_data, normalization_constant = gen_train_data(cfg, opt_model)

In [ ]:
po_model = setup_po_model(
    cfg,
    graph,
    opt_model,
    training_data,
    versatile=True
)

In [ ]:
spo_model = setup_hybrid_spo_model(
    cfg,
    graph,
    opt_model,
    training_data,
    versatile=True,
)

In [ ]:
# # TODO: - Plot cost/est. cost eror
# #       - Create example to see what costs look like here
# import pyepo
# print(f"log_s parameter before Calibrator fitting: {spo_model.log_s.item()}") 
# calibration_trainer = CalibratorTrainer(spo_model)
# calibration_trainer.align_scale_ols(training_data['train_loader'])
# print(f"log_s parameter after Calibrator fitting:  {spo_model.log_s.item()}")
# print(f"New SPO predictor regret:                  {pyepo.metric.regret(spo_model, opt_model, training_data['val_loader'])}")

In [ ]:
from copy import deepcopy
idx = 0
cost = testing_data["costs"][idx]*normalization_constant
feature = testing_data["feats"][idx]
opt_model_temp = deepcopy(opt_model)

# Set the cost for the grid (Optionally specify the source and target nodes)
opt_model_temp.setObj(cost)

# Solve shortest path problem
path, obj = opt_model_temp.solve()

# Predict the shortest path with predict-then-optimize framework
predicted_costs = spo_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy() * normalization_constant
spo_graph = ShortestPathGrid(m, n, cost=predicted_costs)
spo_shortest_path = shortestPathGrb(spo_graph)
spo_path, _ = spo_shortest_path.solve()

# Visualize
fig, axx = plt.subplots(1, 2, figsize=(12,6))
opt_model_temp.visualize(colored_edges=path, title="True Shortest Path", ax=axx[0])
spo_graph.visualize(colored_edges=spo_path, title="SPO Shortest Path", ax=axx[1])
fig.tight_layout()

In [ ]:
true_objs, po_objs, spo_objs = compare_po_spo(cfg, opt_model, po_model, spo_model, testing_data)

In [ ]:
po_val = [(po - true) / true * 100 for po, true in zip(po_objs, true_objs)]
spo_val = [(spo - true) / true * 100 for spo, true in zip(spo_objs, true_objs)]

# plt.scatter(np.ones_like(po_objs), po_val)
# plt.scatter(np.ones_like(spo_objs) + 1, spo_val)

fig, ax = plt.subplots(figsize=(6,4))

# Two groups: PO and SPO
_ = ax.boxplot([po_val, spo_val], tick_labels=['PO', 'SPO'],
           showmeans=True, meanline=True)   # optional styling
ax.set_ylabel("Cost increase [%]")
ax.set_ylim(0,50)
ax.grid()
# plt.legend(['PO', 'SPO'], location='north')

po_mean  = np.mean(po_val)
spo_mean = np.mean(spo_val)

# std devs
po_std_samp = np.std(po_val, ddof=1)
spo_std_samp = np.std(spo_val, ddof=1)
mean_improvement = (po_mean - spo_mean) * 100 / po_mean
std_improvement = (po_std_samp - spo_std_samp) * 100 / po_std_samp

rows = [
    ["Mean",         f"{po_mean:.2f}%",  f"{spo_mean:.2f}%", f"{mean_improvement:.2f}%"],
    ["Std (sample)", f"{po_std_samp:.2f}%", f"{spo_std_samp:.2f}%", f"{std_improvement:.2f}%"],
]
print(tabulate(rows, headers=["Metric", "PO (%)", "SPO (%)", "SPO improvement (%)"], tablefmt="github"))

In [ ]:
interdictions = gen_data(cfg, seed=cfg.get("intd_seed"), normalization_constant=normalization_constant)

In [ ]:
# Comparison of different means of costs to show similar 
print(f"Mean value comparison:")
print(f"----------------------")
print(f"Test:  {testing_data['costs'].mean():.7f}")
print(f"Train: {training_data['train_loader'].dataset.costs.mean():.7f}")
print(f"Intd:  {interdictions['costs'].mean():.7f}")
print(f"PO:    {po_model(torch.tensor(testing_data['feats'], dtype=torch.float32)).mean().item():.7f}")
print(f"SPO+:  {spo_model(torch.tensor(testing_data['feats'], dtype=torch.float32)).mean().item():.7f}")

print(f"\nStd value comparison:")
print(f"----------------------")
print(f"Test:  {testing_data['costs'].std():.7f}")
print(f"Train: {training_data['train_loader'].dataset.costs.std():.7f}")
print(f"Intd:  {interdictions['costs'].std():.7f}")
print(f"PO:    {po_model(torch.tensor(testing_data['feats'], dtype=torch.float32)).std().item():.7f}")
print(f"SPO+:  {spo_model(torch.tensor(testing_data['feats'], dtype=torch.float32)).std().item():.7f}")

In [ ]:
def simulate_interdictor(cfg, test_data, interdictions, normalization_constant, est_model = None, idx = None):
    """
    Compare the performance of the PO and SPO models using symmetric shortest path interdiction.
    This function simulates the interdiction process and evaluates the objective values.
    """

    # Get the number of simulation data samples
    num_test_samples = cfg.get("num_test_samples")

    # Prepare lists to store results
    true_objs = []
    est_objs = []

    # Print that the simulation is starting
    print(f"Running simulation with {num_test_samples} samples...")

    # Iterate through each data sample
    for i in range(num_test_samples) if idx is None else [idx]:
        # Store values for the current sample
        cost = test_data["costs"][i] * normalization_constant
        interdiction = interdictions["costs"][i] * normalization_constant
        m, n = cfg.get("grid_size")

        # Compute estimated cost if an estimator is provided. Otherwise use the true costs
        if est_model is not None:
            feature = test_data["feats"][i]
            est_cost = est_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy() * normalization_constant
        else:
            est_cost = cost

        # Update opt_model
        est_graph = ShortestPathGrid(m, n, cost=est_cost)
        est_opt_model = shortestPathGrb(est_graph)

        # Solutions without information asymmetry
        interdictor_I = BendersDecomposition(est_opt_model, 
                                             k=cfg.get("budget"), 
                                             interdiction_cost=interdiction, 
                                             max_cnt=cfg.get("benders_max_count"), 
                                             eps=cfg.get("benders_eps"))
        x_intd, y_est, _ = interdictor_I.solve(versatile=False if idx is None else True)

        # Create true model
        true_graph = ShortestPathGrid(m, n, cost=cost)
        true_opt_model = shortestPathGrb(true_graph)

        # True shortest path after interdiction
        true_opt_model.setObj(cost + x_intd * interdiction)
        y_true, _ = true_opt_model.solve()

        # Store the results
        true_objs.append(true_graph(y_true, interdictions=x_intd * interdiction))
        est_objs.append(true_graph(y_est, interdictions=x_intd * interdiction))

        # Print progress
        step = max(1, num_test_samples // 20)  # ~every 5% (safe for small N)
        if (i % step == 0) or (i == num_test_samples - 1):
            if i == num_test_samples - 1:
                done, pct = 20, 100                      # final tick
            else:
                done = (i * 20) // num_test_samples      # 0..20 “=”
                pct  = done * 5                          # 0,5,10,...,95

            sys.stdout.write('\r[%-20s] %3d%%' % ('=' * done, pct))
            sys.stdout.flush()
            if i == num_test_samples - 1:
                sys.stdout.write('\n')


    # Evaluate performance
    return {
        "true_objective": np.array(true_objs),
        "est_objective": np.array(est_objs),
    }

# Debugging Bender's Decomposition!

In [ ]:
idx = 0
cost = testing_data["costs"][idx] * normalization_constant
interdiction = interdictions["costs"][idx] * normalization_constant
m, n = cfg.get("grid_size")
est_cost = spo_model(torch.tensor(testing_data["feats"][idx], dtype=torch.float32)).detach().numpy() * normalization_constant

# from models.CalibratedPredictor import scale_alignment_alpha
# scale_alignment_alpha(est_cost, cost)

# Update opt_model
est_graph = ShortestPathGrid(m, n, cost=est_cost)
est_opt_model = shortestPathGrb(est_graph)
est_opt_model.visualize()

In [ ]:
# Solutions without information asymmetry
interdictor_I = BendersDecomposition(est_opt_model, 
                                        k=cfg.get("budget"), 
                                        interdiction_cost=interdiction, 
                                        max_cnt=cfg.get("benders_max_count"), 
                                        eps=cfg.get("benders_eps"))
x_intd, y_est, _ = interdictor_I.solve(versatile=True, visualize=True)

In [ ]:
true_graph = ShortestPathGrid(m, n, cost=cost+x_intd * interdiction)
true_opt_model = shortestPathGrb(true_graph)
y_true, obj = true_opt_model.solve(visualize=True)

# End of Debugging Bender's Decomposition

In [ ]:
idx = 0

In [ ]:
true_interdiction = simulate_interdictor(cfg, testing_data, interdictions, normalization_constant)

In [ ]:
po_interdiction = simulate_interdictor(cfg, testing_data, interdictions, normalization_constant, est_model=po_model)

In [ ]:
spo_interdiction = simulate_interdictor(cfg, testing_data, interdictions, normalization_constant, est_model=spo_model)

In [ ]:
print(true_interdiction['true_objective'].mean())
print(true_interdiction['est_objective'].mean())

In [ ]:
print(po_interdiction['true_objective'].mean())
print(po_interdiction['est_objective'].mean())

In [ ]:
print(spo_interdiction['true_objective'].mean())
print(spo_interdiction['est_objective'].mean())

In [ ]:
idx = 0
for po , true in zip(po_interdiction['true_objective'], true_interdiction['true_objective']):
    if true < po:
        print(idx)
    idx += 1

In [ ]:
po_vals = [(true - po) / true * 100 for po, true in zip(po_interdiction['true_objective'], true_interdiction['true_objective'])]
spo_vals = [(true - spo) / true * 100 for spo, true in zip(spo_interdiction['true_objective'], true_interdiction['true_objective'])]

po_mean  = np.mean(po_vals)
spo_mean = np.mean(spo_vals)

# # std devs
# po_std_pop  = np.std(po_val, ddof=0)   # population σ
# po_std_samp = np.std(po_val, ddof=1)   # sample s (n-1)
# spo_std_pop  = np.std(spo_val, ddof=0)
# spo_std_samp = np.std(spo_val, ddof=1)
# mean_improvement = (po_mean - spo_mean) * 100 / po_mean
# std_improvement = (po_std_samp - spo_std_samp) * 100 / po_std_samp

# Create figure
fig, ax = plt.subplots(figsize=(6,4))

# Two groups: PO and SPO
_ = ax.boxplot([po_vals, spo_vals], tick_labels=['PO', 'SPO'],
           showmeans=True, meanline=True)   # optional styling
ax.set_ylabel("Cost increase [%]")
ax.grid(True)

# std devs
po_std_samp = np.std(po_vals, ddof=1)
spo_std_samp = np.std(spo_vals, ddof=1)
mean_improvement = (po_mean - spo_mean) * 100 / po_mean
std_improvement = (po_std_samp - spo_std_samp) * 100 / po_std_samp

rows = [
    ["Mean",         f"{po_mean:.2f}%",  f"{spo_mean:.2f}%", f"{mean_improvement:.2f}%"],
    ["Std (sample)", f"{po_std_samp:.2f}%", f"{spo_std_samp:.2f}%", f"{std_improvement:.2f}%"],
]
print(tabulate(rows, headers=["Metric", "PO (%)", "SPO (%)", "SPO improvement (%)"], tablefmt="github"))

Takeaway from above: If we post-process the trained cost predictor with SPO, we can achieve a better model and better interdiction (robust) performance!

## Single Examples

In [ ]:
idx = 30

In [ ]:
# Store values for the current sample
feature = testing_data["feats"][idx]
cost = testing_data["costs"][idx] * normalization_constant
interdiction = interdictions["costs"][idx] * normalization_constant
m, n = cfg.get("grid_size")

# Compute estimated cost if an estimator is provided. Otherwise use the true costs
po_cost = po_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy() * normalization_constant
spo_cost = spo_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy() * normalization_constant

# Update estimated opt_models
spo_graph = ShortestPathGrid(m, n, cost=po_cost)
po_opt_model = shortestPathGrb(spo_graph)

spo_graph = ShortestPathGrid(m, n, cost=spo_cost)
spo_opt_model = shortestPathGrb(spo_graph)

# Create true model
true_graph = ShortestPathGrid(m, n, cost=cost)
true_opt_model = shortestPathGrb(true_graph)

# Display estimated and true shortest paths and costs
fig, ax = plt.subplots(1, 3, figsize=(18, 6))
y_true, _ = true_opt_model.solve(visualize=True, ax=ax[0], title="True Shortest Path")
y_est, _ = po_opt_model.solve(visualize=True, ax=ax[1], title="PO Shortest Path")
y_spo, _ = spo_opt_model.solve(visualize=True, ax=ax[2], title="SPO Shortest Path")
fig.tight_layout()
print(f"True path cost: {true_graph(y_true):.3f}")
print(f"PO path cost: {true_graph(y_est):.3f}")
print(f"SPO path cost: {true_graph(y_spo):.3f}")

In [ ]:
# Solutions without information asymmetry
true_interdictor = BendersDecomposition(true_opt_model, 
                                        k=cfg.get("budget"), 
                                        interdiction_cost=interdiction, 
                                        max_cnt=cfg.get("benders_max_count"), 
                                        eps=cfg.get("benders_eps"))
x_true, y_est_true, _ = true_interdictor.solve(versatile=False)

po_interdictor = BendersDecomposition(po_opt_model, 
                                        k=cfg.get("budget"), 
                                        interdiction_cost=interdiction, 
                                        max_cnt=cfg.get("benders_max_count"), 
                                        eps=cfg.get("benders_eps"))
x_po, y_est_po, _ = po_interdictor.solve(versatile=False)

spo_interdictor = BendersDecomposition(spo_opt_model, 
                                        k=cfg.get("budget"), 
                                        interdiction_cost=interdiction, 
                                        max_cnt=cfg.get("benders_max_count"), 
                                        eps=cfg.get("benders_eps"))
x_spo, y_est_spo, _ = spo_interdictor.solve(versatile=False)

# True shortest paths after interdiction
true_opt_model.setObj(cost + x_true * interdiction)
y_true_true, _ = true_opt_model.solve()

true_opt_model.setObj(cost + x_po * interdiction)
y_true_po, _ = true_opt_model.solve()

true_opt_model.setObj(cost + x_spo * interdiction)
y_true_spo, _ = true_opt_model.solve()

# Store the results
# true_graph(y_true, interdictions=x_intd * interdiction)
# true_graph(y_est, interdictions=x_intd * interdiction)

In [ ]:
# Create figure for plotting
fig, ax = plt.subplots(2, 3, figsize=(18, 12))

# Estimated shortest paths after interdiction
true_graph.visualize(ax=ax[0, 0], colored_edges = y_est_true, dashed_edges=x_true, title="True Intd - Estimated Path")

spo_graph.visualize(ax=ax[0, 1], colored_edges = y_est_po, dashed_edges=x_po, title="PO Intd - Estimated Path")

spo_graph.visualize(ax=ax[0, 2], colored_edges = y_est_spo, dashed_edges=x_spo, title="SPO Intd - Estimated Path")

# True shortest paths after interdiction
true_graph.visualize(ax=ax[1, 0], colored_edges = y_true_true, dashed_edges=x_true, title="True Intd - True Path")

spo_graph.visualize(ax=ax[1, 1], colored_edges = y_true_po, dashed_edges=x_po, title="PO Intd - True Path")

spo_graph.visualize(ax=ax[1, 2], colored_edges = y_true_spo, dashed_edges=x_spo, title="SPO Intd - True Path")

fig.tight_layout()

# Print results
print(f"{true_graph(y_est_true, interdictions=x_true * interdiction)} | {true_graph(y_est_po, interdictions=x_po * interdiction)} | {true_graph(y_est_spo, interdictions=x_spo * interdiction)}")
print(f"{true_graph(y_true_true, interdictions=x_true * interdiction)} | {true_graph(y_true_po, interdictions=x_po * interdiction)} | {true_graph(y_true_spo, interdictions=x_spo * interdiction)}")

In [ ]:
# Estimated shortest paths after interdiction
true_graph.visualize(colored_edges = y_est_true, dashed_edges=x_true, title="True Intd - Estimated Path")

In [ ]:
import numpy as np
xon = np.array([4.2703, 3.9413, 4.0144, 3.8332, 4.7098, 5.4733, 5.0940, 4.2866, 4.6499, 4.6499])
print(f"Oracle & no intd.: {xon.mean()} +/- {xon.std()}")
xos = np.array([6.8659, 6.7464, 6.4299, 6.1716, 7.4298, 7.5481, 6.9647, 6.6544, 7.2118, 7.2118])
print(f"Oracle & sym intd: {xos.mean()} +/- {xos.std()}")
xoa = np.array([6.8728, 7.2865, 6.9242, 6.1716, 7.4298, 7.5481, 7.5056, 6.6649, 7.2118, 7.2118])
print(f"Oracle & asym intd: {xoa.mean()} +/- {xoa.std()}")
xpn = np.array([4.7798, 4.1016, 4.1909, 4.2144, 5.1734, 6.0128, 5.2856, 4.8201, 4.8097, 4.8097])
print(f"PO & no intd: {xpn.mean()} +/- {xpn.std()}")
xps = np.array([7.2351, 6.9740, 6.6728, 6.6493, 7.9041, 8.2390, 7.7915, 7.1620, 7.3979, 7.3979])
print(f"PO & sym intd: {xps.mean()} +/- {xps.std()}")
xpa = np.array([8.0121, 5.7358, 6.9242, 7.1088, 8.3745, 9.1074, 8.2071, 7.9188, 7.7143, 7.7143])
print(f"PO & asym intd: {xpa.mean()} +/- {xpa.std()}")
xsn = np.array([4.3636, 4.0688, 4.1207, 3.9578, 4.8607, 5.5872, 5.2111, 4.3534, 4.7325, 4.7325])
print(f"SPO & no intd: {xsn.mean()} +/- {xsn.std()}")
xss = np.array([7.4021, 7.1997, 6.7817, 6.6076, 7.9676, 7.9900, 8.0484, 7.2554, 7.6427, 7.6427])
print(f"SPO & sym intd: {xss.mean()} +/- {xss.std()}")
xsa = np.array([8.8781, 8.1814, 7.7326, 7.6505, 9.0112, 9.2008, 9.3887, 8.7232, 8.6073, 8.6073])
print(f"SPO & asym intd: {xsa.mean()} +/- {xsa.std()}")
xan = np.array([4.4482, 4.1223, 4.1533, 3.9876, 4.9809, 5.6577, 5.2432, 4.4143, 4.7514, 4.7514])
print(f"adv SPO & no intd: {xan.mean()} +/- {xan.std()}")
xas = np.array([7.1498, 7.0349, 6.6749, 6.4627, 7.7634, 7.8205, 7.8618, 7.0229, 7.4465, 7.4465])
print(f"adv SPO & sym intd: {xas.mean()} +/- {xas.std()}")
xaa = np.array([7.7773, 7.4520, 7.1286, 6.9911, 8.1931, 8.4785, 8.4053, 7.7172, 7.9356, 7.9356])
print(f"adv SPO & asym intd: {xaa.mean()} +/- {xaa.std()}")

In [ ]:
xon

In [ ]:
xon = 4.2703 + 3.9413 + 4.0144 + 3.8332 + 4.7098 + 5.4733 + 5.0940 + 4.2866 + 4.6499 + 4.6499
print(f"Oracle & no intd.: {xon}")
xos = 6.8659 + 6.7464 + 6.4299 + 6.1716 + 7.4298 + 7.5481 + 6.9647 + 6.6544 + 7.2118 + 7.2118
print(f"Oracle & sym intd: {xos}")
xoa = 6.8728 + 7.2865 + 6.9242 + 6.1716 + 7.4298 + 7.5481 + 7.5056 + 6.6649 + 7.2118 + 7.2118
print(f"Oracle & asym intd: {xoa}")
xpn = 4.7798 + 4.1016 + 4.1909 + 4.2144 + 5.1734 + 6.0128 + 5.2856 + 4.8201 + 4.8097 + 4.8097
print(f"PO & no intd: {xpn}")
xps = 7.2351 + 6.9740 + 6.6728 + 6.6493 + 7.9041 + 8.2390 + 7.7915 + 7.1620 + 7.3979 + 7.3979
print(f"PO & sym intd: {xps}")
xpa = 8.0121 + 5.7358 + 6.9242 + 7.1088 + 8.3745 + 9.1074 + 8.2071 + 7.9188 + 7.7143 + 7.7143
print(f"PO & asym intd: {xpa}")
xsn = 4.3636 + 4.0688 + 4.1207 + 3.9578 + 4.8607 + 5.5872 + 5.2111 + 4.3534 + 4.7325 + 4.7325
print(f"SPO & no intd: {xsn}")
xss = 7.4021 + 7.1997 + 6.7817 + 6.6076 + 7.9676 + 7.9900 + 8.0484 + 7.2554 + 7.6427 + 7.6427
print(f"SPO & sym intd: {xss}")
xsa = 8.8781 + 8.1814 + 7.7326 + 7.6505 + 9.0112 + 9.2008 + 9.3887 + 8.7232 + 8.6073 + 8.6073
print(f"SPO & asym intd: {xsa}")
xan = 4.4482 + 4.1223 + 4.1533 + 3.9876 + 4.9809 + 5.6577 + 5.2432 + 4.4143 + 4.7514 + 4.7514
print(f"adv SPO & no intd: {xan}")
xas = 7.1498 + 7.0349 + 6.6749 + 6.4627 + 7.7634 + 7.8205 + 7.8618 + 7.0229 + 7.4465 + 7.4465
print(f"adv SPO & sym intd: {xas}")
xaa = 7.7773 + 7.4520 + 7.1286 + 6.9911 + 8.1931 + 8.4785 + 8.4053 + 7.7172 + 7.9356 + 7.9356
print(f"adv SPO & asym intd: {xaa}")